## File Reading

In [4]:
# Read CSV with Pandas and Check RunTime
import pandas as pd
import time
st = time.time()
pandas_df = pd.read_csv('used_cars_data.csv')
et = time.time()
tt = et - st
print('Pandas execution time:', tt, 'seconds')

C:\Users\ammar\AppData\Local\Temp\ipykernel_4648\1283812198.py:5: DtypeWarning: Columns (11) have mixed types. Specify dtype option on import or set low_memory=False.
  pandas_df = pd.read_csv('used_cars_data.csv')


Pandas execution time: 163.3865180015564 seconds


In [5]:
# Read CSV with Dask and Check RunTime
from dask import dataframe as dd
import time
st = time.time()
dask_df = dd.read_csv('used_cars_data.csv')
et = time.time()
tt = et - st
print('Dask execution time:', tt, 'seconds')

Dask execution time: 0.15074634552001953 seconds


In [ ]:
# Read CSV with Modin/Ray and Check Runtime
import modin.pandas as pd
import ray
ray.shutdown()
ray.init()
import time
st = time.time()
modin_ray_df = pd.read_csv('used_cars_data.csv')
et = time.time()
tt = et - st
print('Modin/Ray execution time:', tt, 'seconds')

2022-11-11 11:55:11,401	INFO worker.py:1509 -- Started a local Ray instance. View the dashboard at 127.0.0.1:8265 


**Dask** is by far the fastest method of reading the large dataset (~0.15 seconds) with the most computational efficiency. 

## Data Validation

In [2]:
import pandas as pd
df = pd.read_csv('used_cars_data.csv', delimiter = ',')
df.head(5)

C:\Users\ammar\AppData\Local\Temp\ipykernel_10480\392060150.py:2: DtypeWarning: Columns (11) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('used_cars_data.csv', delimiter = ',')


,vin,back_legroom,bed,bed_height,bed_length,body_type,cabin,city,city_fuel_economy,combine_fuel_economy,...,transmission,transmission_display,trimId,trim_name,vehicle_damage_category,wheel_system,wheel_system_display,wheelbase,width,year
0,ZACNJABB5KPJ92081,35.1 in,NaN,NaN,NaN,SUV / Crossover,NaN,Bayamon,NaN,NaN,...,A,9-Speed Automatic Overdrive,t83804,Latitude FWD,NaN,FWD,Front-Wheel Drive,101.2 in,79.6 in,2019
1,SALCJ2FX1LH858117,38.1 in,NaN,NaN,NaN,SUV / Crossover,NaN,San Juan,NaN,NaN,...,A,9-Speed Automatic Overdrive,t86759,S AWD,NaN,AWD,All-Wheel Drive,107.9 in,85.6 in,2020
2,JF1VA2M67G9829723,35.4 in,NaN,NaN,NaN,Sedan,NaN,Guaynabo,17.0,NaN,...,M,6-Speed Manual,t58994,Base,NaN,AWD,All-Wheel Drive,104.3 in,78.9 in,2016
3,SALRR2RV0L2433391,37.6 in,NaN,NaN,NaN,SUV / Crossover,NaN,San Juan,NaN,NaN,...,A,8-Speed Automatic Overdrive,t86074,V6 HSE AWD,NaN,AWD,All-Wheel Drive,115 in,87.4 in,2020
4,SALCJ2FXXLH862327,38.1 in,NaN,NaN,NaN,SUV / Crossover,NaN,San Juan,NaN,NaN,...,A,9-Speed Automatic Overdrive,t86759,S AWD,NaN,AWD,All-Wheel Drive,107.9 in,85.6 in,2020


In [3]:
# Remove White Spaces
df.columns = df.columns.str.replace(' ', '_')

# Remove Special Characters
df.columns = df.columns.str.replace('[?, ~, -, +, $, ^, *, #, @, &]', '')

# Inspect Columns
df.columns

C:\Users\ammar\AppData\Local\Temp\ipykernel_10480\2125231692.py:5: FutureWarning: The default value of regex will change from True to False in a future version.
  df.columns = df.columns.str.replace('[?, ~, -, +, $, ^, *, #, @, &]', '')


Index(['vin', 'back_legroom', 'bed', 'bed_height', 'bed_length', 'body_type',
       'cabin', 'city', 'city_fuel_economy', 'combine_fuel_economy',
       'daysonmarket', 'dealer_zip', 'description', 'engine_cylinders',
       'engine_displacement', 'engine_type', 'exterior_color', 'fleet',
       'frame_damaged', 'franchise_dealer', 'franchise_make', 'front_legroom',
       'fuel_tank_volume', 'fuel_type', 'has_accidents', 'height',
       'highway_fuel_economy', 'horsepower', 'interior_color', 'isCab',
       'is_certified', 'is_cpo', 'is_new', 'is_oemcpo', 'latitude', 'length',
       'listed_date', 'listing_color', 'listing_id', 'longitude',
       'main_picture_url', 'major_options', 'make_name', 'maximum_seating',
       'mileage', 'model_name', 'owner_count', 'power', 'price', 'salvage',
       'savings_amount', 'seller_rating', 'sp_id', 'sp_name', 'theft_title',
       'torque', 'transmission', 'transmission_display', 'trimId', 'trim_name',
       'vehicle_damage_category', 'whe

## Data Ingestion
1. Utility file creation.
2. Config file creation. 
3. Data ingestion pipeline. 

In [4]:
import logging
import os
import subprocess
import yaml
import pandas as pd
import datetime 
import gc
import re

In [5]:
%%writefile utility.py
import logging
import os
import subprocess
import yaml
import pandas as pd
import datetime 
import gc
import re

# File Reading
def read_config_file(filepath):
    with open(filepath, 'r') as stream:
        try:
            return yaml.safe_load(stream)
        except yaml.YAMLError as exc: 
            logging.error(exec)

def replacer(string, char):
    pattern = char + '{2,}'
    string = re.sub(pattern, char, string)
    return string

def col_header_val(df, table_config):
    '''
    Replace whitespaces in the column
    and standardized column names.
    '''
    df.columns = df.columns.str.lower()
    df.columns = df.columns.str.replace('[^\w]','_',regex=True)
    df.columns = list(map(lambda x: x.strip('_'), list(df.columns)))
    df.columns = list(map(lambda x: replacer(x, '_'), list(df.columns)))
    expected_col = list(map(lambda x: x.lower(),  table_config['columns']))
    expected_col.sort()
    df.columns =list(map(lambda x: x.lower(), list(df.columns)))
    df = df.reindex(sorted(df.columns), axis=1)
    if len(df.columns) == len(expected_col) and list(expected_col)  == list(df.columns):
        print("column name and column length validation passed")
        return 1
    else:
        print("column name and column length validation failed")
        mismatched_columns_file = list(set(df.columns).difference(expected_col))
        print("Following File columns are not in the YAML file",mismatched_columns_file)
        missing_YAML_file = list(set(expected_col).difference(df.columns))
        print("Following YAML columns are not in the file uploaded",missing_YAML_file)
        logging.info(f'df columns: {df.columns}')
        logging.info(f'expected columns: {expected_col}')
        return 0

Overwriting utility.py


## Write YAML File

In [12]:
%%writefile file.yaml
file_type: csv
dataset_name: file
file_name: used_cars_data
table_name: edsurv
inbound_delimiter: ","
outbound_delimiter: "|"
skip_leading_rows: 1
columns:
    - vin
    - back_legroom
    - bed
    - bed_height
    - bed_length
    - body_type
    - cabin
    - city
    - city_fuel_economy
    - combine_fuel_economy
    - daysonmarket
    - dealer_zip
    - description
    - engine_cylinders
    - engine_displacement
    - engine_type
    - exterior_color
    - fleet
    - frame_damaged
    - franchise_dealer
    - franchise_make
    - front_legroom
    - fuel_tank_volume
    - fuel_type 
    - has_accidents
    - height
    - highway_fuel_economy
    - horsepower 
    - interior_color 
    - isCab
    - is_certified
    - is_cpo 
    - is_new 
    - is_oemcpo 
    - latitude 
    - length
    - listed_date 
    - listing_color 
    - listing_id 
    - longitude
    - main_picture_url
    - major_options 
    - make_name 
    - maximum_seating
    - mileage 
    - model_name 
    - owner_count 
    - power 
    - price 
    - salvage
    - savings_amount
    - seller_rating 
    - sp_id 
    - sp_name 
    - theft_title
    - torque 
    - transmission 
    - transmission_display 
    - trimId 
    - trim_name
    - vehicle_damage_category 
    - wheel_system 
    - wheel_system_display
    - wheelbase 
    - width 
    - year

Overwriting file.yaml


In [13]:
# Read config file
import utility as util
config_data = util.read_config_file("file.yaml")

In [14]:
config_data['inbound_delimiter']

','

In [15]:
# Inspectin Config File Data
config_data

{'file_type': 'csv',
 'dataset_name': 'file',
 'file_name': 'used_cars_data',
 'table_name': 'edsurv',
 'inbound_delimiter': ',',
 'outbound_delimiter': '|',
 'skip_leading_rows': 1,
 'columns': ['vin',
  'back_legroom',
  'bed',
  'bed_height',
  'bed_length',
  'body_type',
  'cabin',
  'city',
  'city_fuel_economy',
  'combine_fuel_economy',
  'daysonmarket',
  'dealer_zip',
  'description',
  'engine_cylinders',
  'engine_displacement',
  'engine_type',
  'exterior_color',
  'fleet',
  'frame_damaged',
  'franchise_dealer',
  'franchise_make',
  'front_legroom',
  'fuel_tank_volume',
  'fuel_type',
  'has_accidents',
  'height',
  'highway_fuel_economy',
  'horsepower',
  'interior_color',
  'isCab',
  'is_certified',
  'is_cpo',
  'is_new',
  'is_oemcpo',
  'latitude',
  'length',
  'listed_date',
  'listing_color',
  'listing_id',
  'longitude',
  'main_picture_url',
  'major_options',
  'make_name',
  'maximum_seating',
  'mileage',
  'model_name',
  'owner_count',
  'power',
  

In [30]:
# Read the File with Pandas
df_pandas = pd.read_csv('used_cars_data.csv', delimiter = ',')
df_pandas.head(5)

C:\Users\ammar\AppData\Local\Temp\ipykernel_5568\2618117214.py:2: DtypeWarning: Columns (11) have mixed types. Specify dtype option on import or set low_memory=False.
  df_pandas = pd.read_csv('used_cars_data.csv', delimiter = ',')


,vin,back_legroom,bed,bed_height,bed_length,body_type,cabin,city,city_fuel_economy,combine_fuel_economy,...,transmission,transmission_display,trimId,trim_name,vehicle_damage_category,wheel_system,wheel_system_display,wheelbase,width,year
0,ZACNJABB5KPJ92081,35.1 in,NaN,NaN,NaN,SUV / Crossover,NaN,Bayamon,NaN,NaN,...,A,9-Speed Automatic Overdrive,t83804,Latitude FWD,NaN,FWD,Front-Wheel Drive,101.2 in,79.6 in,2019
1,SALCJ2FX1LH858117,38.1 in,NaN,NaN,NaN,SUV / Crossover,NaN,San Juan,NaN,NaN,...,A,9-Speed Automatic Overdrive,t86759,S AWD,NaN,AWD,All-Wheel Drive,107.9 in,85.6 in,2020
2,JF1VA2M67G9829723,35.4 in,NaN,NaN,NaN,Sedan,NaN,Guaynabo,17.0,NaN,...,M,6-Speed Manual,t58994,Base,NaN,AWD,All-Wheel Drive,104.3 in,78.9 in,2016
3,SALRR2RV0L2433391,37.6 in,NaN,NaN,NaN,SUV / Crossover,NaN,San Juan,NaN,NaN,...,A,8-Speed Automatic Overdrive,t86074,V6 HSE AWD,NaN,AWD,All-Wheel Drive,115 in,87.4 in,2020
4,SALCJ2FXXLH862327,38.1 in,NaN,NaN,NaN,SUV / Crossover,NaN,San Juan,NaN,NaN,...,A,9-Speed Automatic Overdrive,t86759,S AWD,NaN,AWD,All-Wheel Drive,107.9 in,85.6 in,2020


In [16]:
# read the file using config file
file_type = config_data['file_type']
source_file = "./" + config_data['file_name'] + f'.{file_type}'
df = pd.read_csv(source_file,config_data['inbound_delimiter'])
df.head()

C:\Users\ammar\AppData\Local\Temp\ipykernel_10480\4159071769.py:4: FutureWarning: In a future version of pandas all arguments of read_csv except for the argument 'filepath_or_buffer' will be keyword-only.
  df = pd.read_csv(source_file,config_data['inbound_delimiter'])
C:\Users\ammar\AppData\Local\Temp\ipykernel_10480\4159071769.py:4: DtypeWarning: Columns (11) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(source_file,config_data['inbound_delimiter'])


,vin,back_legroom,bed,bed_height,bed_length,body_type,cabin,city,city_fuel_economy,combine_fuel_economy,...,transmission,transmission_display,trimId,trim_name,vehicle_damage_category,wheel_system,wheel_system_display,wheelbase,width,year
0,ZACNJABB5KPJ92081,35.1 in,NaN,NaN,NaN,SUV / Crossover,NaN,Bayamon,NaN,NaN,...,A,9-Speed Automatic Overdrive,t83804,Latitude FWD,NaN,FWD,Front-Wheel Drive,101.2 in,79.6 in,2019
1,SALCJ2FX1LH858117,38.1 in,NaN,NaN,NaN,SUV / Crossover,NaN,San Juan,NaN,NaN,...,A,9-Speed Automatic Overdrive,t86759,S AWD,NaN,AWD,All-Wheel Drive,107.9 in,85.6 in,2020
2,JF1VA2M67G9829723,35.4 in,NaN,NaN,NaN,Sedan,NaN,Guaynabo,17.0,NaN,...,M,6-Speed Manual,t58994,Base,NaN,AWD,All-Wheel Drive,104.3 in,78.9 in,2016
3,SALRR2RV0L2433391,37.6 in,NaN,NaN,NaN,SUV / Crossover,NaN,San Juan,NaN,NaN,...,A,8-Speed Automatic Overdrive,t86074,V6 HSE AWD,NaN,AWD,All-Wheel Drive,115 in,87.4 in,2020
4,SALCJ2FXXLH862327,38.1 in,NaN,NaN,NaN,SUV / Crossover,NaN,San Juan,NaN,NaN,...,A,9-Speed Automatic Overdrive,t86759,S AWD,NaN,AWD,All-Wheel Drive,107.9 in,85.6 in,2020


In [17]:
# Validate the Header of the File
util.col_header_val(df, config_data)

column name and column length validation passed


1

In [18]:
# Compare columns of both files
print("columns of files are:" , df.columns)
print("columns of YAML are:" , config_data['columns'])

columns of files are: Index(['vin', 'back_legroom', 'bed', 'bed_height', 'bed_length', 'body_type',
       'cabin', 'city', 'city_fuel_economy', 'combine_fuel_economy',
       'daysonmarket', 'dealer_zip', 'description', 'engine_cylinders',
       'engine_displacement', 'engine_type', 'exterior_color', 'fleet',
       'frame_damaged', 'franchise_dealer', 'franchise_make', 'front_legroom',
       'fuel_tank_volume', 'fuel_type', 'has_accidents', 'height',
       'highway_fuel_economy', 'horsepower', 'interior_color', 'iscab',
       'is_certified', 'is_cpo', 'is_new', 'is_oemcpo', 'latitude', 'length',
       'listed_date', 'listing_color', 'listing_id', 'longitude',
       'main_picture_url', 'major_options', 'make_name', 'maximum_seating',
       'mileage', 'model_name', 'owner_count', 'power', 'price', 'salvage',
       'savings_amount', 'seller_rating', 'sp_id', 'sp_name', 'theft_title',
       'torque', 'transmission', 'transmission_display', 'trimid', 'trim_name',
       'vehicle_

In [20]:
# Check to see if column validation 
if util.col_header_val(df,config_data)==0:
    print("validation failed")
    # write code to reject the file
else:
    print("col validation passed")

column name and column length validation passed
col validation passed


In [21]:
# Write File in gz Format with Pipe Seperated Text
df.to_csv('used_cars_data.csv.gz', sep = '|', compression = 'gzip')

## File Summary

In [22]:
# Total Number of Rows
count_row = df.shape[0]
print("The total number of rows in the file are: " + str(count_row))

The total number of rows in the file are: 3000040


In [23]:
# Total Number of Columns
count_col = df.shape[1]
print("The total number of columns in the file are: " + str(count_col))

The total number of columns in the file are: 66


In [24]:
# File Size
import os.path
file_path = r'C:/Users/ammar/documents/sample_project_1/used_cars_data.csv.gz'
file_size = os.path.getsize(file_path)
print(f'The {file_path} size is', file_size, 'bytes')

The C:/Users/ammar/documents/sample_project_1/used_cars_data.csv.gz size is 2243201166 bytes
